In [1]:
#import BeautifulSoup

import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
# Request from URL
r = requests.get('http://www.indeed.com/jobs?q=data+scientist&l=New+York+State&start=70')
r

<Response [200]>

In [3]:
#print headers and content

print r.headers
print r.content

{'X-Content-Type-Options': 'nosniff', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'CTK=1ave6qnmqa56v9op; Domain=.indeed.com; Expires=Mon, 24-Oct-2033 05:00:48 GMT; Path=/, ctkgen=1; Expires=Wed, 19-Oct-2016 10:15:17 GMT; Path=/, JSESSIONID=846433A3E4873B7E968F276865CC514C.jasxA_ord-job38; Path=/; HttpOnly, INDEED_CSRF_TOKEN=JsHzd7qGWyGEwxlteaLwSqRWXX5pdarD; Path=/, PREF=""; Domain=.indeed.com; Expires=Thu, 01-Jan-1970 00:00:10 GMT; Path=/, PREF="TM=1476871937756:L=New+York+State"; Version=1; Max-Age=536870911; Expires=Mon, 24-Oct-2033 05:00:48 GMT; Path=/, RQ="q=data+scientist&l=New+York+State"; Version=1; Max-Age=2592000; Expires=Fri, 18-Nov-2016 10:12:17 GMT; Path=/, UD="LA=1476871937:CV=1476871937:TS=1476871937:SG=dac3f075029a522259cb18f5209a7605"; Version=1; Max-Age=536870911; Expires=Mon, 24-Oct-2033 05:00:48 GMT; Path=/, BIGipServerjob=!it1zLRiAENT8N0c/0rl98CFw+W0yBl8u81H7EAn35mieQZXzDbGRRis9gnP8i66R+XTc8Enj3qmwQ/M=; path=/', 'Vary': 'User-Agent,Acce

In [4]:
# Use BeautifulSoup

insoup = BeautifulSoup(r.content,'lxml')

In [5]:
# function for getting location from indeed

def extract_location_from_result(loc):
    try:
        for n in loc.findAll(class_='location'):
            return n.text.strip()
    except:
        return 'None'

In [6]:
extract_location_from_result(insoup)

u'New York, NY'

In [7]:
# function for getting job title from indeed

def extract_jobtitle_from_result(jobtitle):
    try:
        for n in jobtitle.findAll(class_='jobtitle'):
            return n.text.strip()
    except:
        return 'None' 

In [8]:
extract_jobtitle_from_result(insoup)

u'Machine Learning Quantitative Analyst'

In [9]:
# function for getting company from indeed

def extract_company_from_result(comp):
    try:
        for n in comp.findAll(class_='company'):
            return n.text.strip()
    except:
        return 'None' 

In [10]:
extract_company_from_result(insoup)

u'Bloomberg'

In [11]:
# function to get salary from indeed

def extract_salary_from_result(sal):
    try:
        for n in sal.findAll('td', class_='snip'):
            return n.nobr
    except:
        return 'None'  

In [12]:
extract_salary_from_result(insoup)

In [13]:
# function to get summary/description from indeed

def extract_summary_from_result(desc):
    try:
        for n in desc.findAll('span', class_='summary'):
            return n.text.strip()
    except:
        return 'None' 

In [14]:
extract_summary_from_result(insoup)

u'The Machine Learning Quantitative Analyst will work in Bloomberg\u2019s Enterprise Solutions area and work collaboratively to build a liquidity tool for banks,...'

In [15]:
#Get each list from each page and 

Location = []
Job_Title = []
Company = []
Salary = []
Description = []

url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100
cities = ['New+York', 'Chicago', 'Boston', 'San+Francisco', 'Los+Angeles', 'Austin', 'Atlanta']
results = []
for city in set(cities):
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(city,start)
        ru = requests.get(url)
        indeed = BeautifulSoup(ru.content, "lxml")
        for item in indeed.findAll('div', class_='result'):
            Location.append(extract_location_from_result(item))
            Job_Title.append(extract_jobtitle_from_result(item))
            Company.append(extract_company_from_result(item))
            Salary.append(extract_salary_from_result(item))
            Description.append(extract_summary_from_result(item))

In [16]:
import pandas as pd

In [17]:
df = pd.DataFrame([Location,Job_Title,Company,Salary,Description]).T 

In [18]:
df.columns= ['Location','Job_Title','Company','Salary','Description']

In [19]:
df.head()

,Location,Job_Title,Company,Salary,Description
0,"Austin, TX 78730",Data Scientist,Cognitive Scale,None,This position is specifically for people with ...
1,"Austin, TX 78730",Software Engineer - Machine Learning,Cognitive Scale,None,The convergence of data and technology is tran...
2,"Austin, TX 78731",Data Engineer,Indeed,None,Work closely with stakeholders on the data dem...
3,"Austin, TX",Research Scientist,University of Texas at Austin,"<nobr>$4,584 a month</nobr>",Provide statistical support on research and su...
4,"Austin, TX",Staff Data Scientist,HomeAway,None,Experience in processing and analyzing Big dat...


In [20]:
#Cleaning Salary Column

df['Salary'].unique()

array([None, <nobr>$4,584 a month</nobr>,
       <nobr>$6,250 - $10,833 a month</nobr>,
       <nobr>$4,599 - $6,066 a month</nobr>, <nobr>$6,667 a month</nobr>,
       <nobr>$5,541 a month</nobr>, <nobr>$5,400 - $6,500 a month</nobr>,
       <nobr>$4,000 a month</nobr>, <nobr>$3,520 - $4,600 a month</nobr>,
       <nobr>$4,916 a month</nobr>, <nobr>$5,259 - $8,624 a month</nobr>,
       <nobr>$140,000 - $160,000 a year</nobr>,
       <nobr>$23.44 - $29.80 an hour</nobr>,
       <nobr>$5,259 - $6,941 a month</nobr>,
       <nobr>$4,500 - $4,900 a month</nobr>,
       <nobr>$120,000 - $150,000 a year</nobr>,
       <nobr>$120,000 a year</nobr>,
       <nobr>$80,000 - $110,000 a year</nobr>,
       <nobr>$180,000 a year</nobr>, <nobr>$110,000 a year</nobr>,
       <nobr>$104,674 - $151,358 a year</nobr>,
       <nobr>$105,000 a year</nobr>,
       <nobr>$135,000 - $165,000 a year</nobr>,
       <nobr>$100,000 - $150,000 a year</nobr>, <nobr>$1,250 a week</nobr>,
       <nobr>$34,142 a ye

In [21]:
# Remove columns with 'None' in Salary

df = df[df.Salary.notnull()]

In [22]:
#Convert Salary column to a string

df['Salary'] = df['Salary'].apply(lambda x: str(x))

In [23]:
# Remove $ and , in Salary

df['Salary'] = df['Salary'].apply(lambda x: x.replace('$',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace(',',''))

In [24]:
# Remove nobr tags from Salary

df['Salary'] = df['Salary'].apply(lambda x: x.replace('<nobr>',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace('</nobr>',''))

In [29]:
# Reset index

df = df.reset_index(drop=True)

In [31]:
# Parse yearly, monthly, hourly salaries

df['hourly_sal'] = df['Salary'].map(lambda x: 1 if 'hour' in x else 0)
df['monthly_sal'] = df['Salary'].map(lambda x: 1 if 'month' in x else 0)
df['yearly_sal'] = df['Salary'].map(lambda x: 1 if 'year' in x else 0)


In [33]:
# Drop monthly and hourly salaries
df_year1 = df[df.monthly_sal != 1]
df_year = df_year1[df.hourly_sal !=1]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [35]:
# drop columns hourly_sal, monthly_sal, and yearly_sal
df_year = df_year.drop('hourly_sal', axis=1)
df_year = df_year.drop('monthly_sal', axis=1)
df_year = df_year.drop('yearly_sal', axis=1)

In [38]:
# reset index and set dataframe to df
df = df_year.reset_index(drop=True)

In [42]:
# remove a year and - in Salary column
df["Salary"] = df["Salary"].apply(lambda x: x.replace(" a year",""))
df["Salary"] = df["Salary"].apply(lambda x: x.replace("-",""))


In [62]:
df

,Location,Job_Title,Company,Salary,Description
0,"Austin, TX",Senior Machine Learning Data Scientist,All-In Analytics,NaN,"Machine Learning Data Scientist. Forecasting, ..."
1,"Austin, TX",Machine Learning Engineer,Volt Workforce Solutions,NaN,Data mining competition experience preferred (...
2,"Chicago, IL",Data Scientist -Fitness/Wellness Firm,Hirewell,120000.0,Bring structure to large quantities of data - ...
3,"Chicago, IL",Data Scientist,Workbridge Associates,NaN,Big data with Hadoop is a plus. A well-respect...
4,"Chicago, IL",Quantitative Research Analyst,GinasTechJobs.com,180000.0,Define and implement data collection and data ...
5,"Chicago, IL",Data Scientist (Healthcare Data),Workbridge Associates,110000.0,Ability to work with unstructured healthcare d...
6,"Chicago, IL",Supervising Environmental Soil Scientist,Metropolitan Water Reclamation District of Gre...,NaN,"Collection, analysis, interpretation and prese..."
7,"Chicago, IL",Mid-Level Data Scientist,Workbridge Associates,105000.0,"Data, Analytics, Scientist, Science, Python, R..."
8,"Chicago, IL",Data Scientist – Big Data/Artificial Intelligence,Selby Jennings,NaN,Data Scientist – Big Data/Artificial Intellige...
9,"San Francisco, CA",Data Scientist,Workbridge Associates,NaN,MS in similar program with 2+ years of experie...


In [54]:
# Find mean of salary with more than one salary

def mean_salary(salary):
    if salary.split(' ')
    .apply()
    else:
        return salary

dtype('float64')

In [ ]:
df['Salary'] = pd.to_numeric(df['Salary'], errors = 'coerce')
df['Salary'].dtype